# Commitments

In this chapter we look at cryptographic commitments.

This primitive is used basically all interactive proofs because it is so useful.

# TL;DR

A commitment is an encrypted value. The creator of the commitment can decrypt ("open") the commitment.

The commitment does not reveal anything about its contained value (hiding property).

The commitment can only be opened to the value that was originally encrypted (binding property).

Usually Peggy makes a large number of commitments and sends them to Victor. These commitments cover all possible cases of the statement to be proved. Victor randomly chooses a case. Peggy responds by opening a small number of corresponding commitments.

Because of the setup, Peggy has to be able to respond to all possible cases. If Peggy cannot respond to one case, then Victor might choose that one. Because of the commitments, Peggy cannot undo what she sent and Victor will expose her lying.

This saves bandwidth because only one case out of all possible cases is sent over the wire.

# Jupyter setup

Run the following snippet to set up your jupyter notebook for the workshop.

In [ ]:
import sys

# Add project root so we can import local modules
root_dir = sys.path.append("..")
sys.path.append(root_dir)

# Import here so cells don't depend on each other
from typing import List, Tuple
from local.ec.util import Opening
from local.ec.static import Scalar, CurvePoint, ONE_POINT
import random

# Pedersen commitments (attempt)

Pedersen commitments enable us to commit to scalar values on an elliptic curve.

The maximum amount of information inside a value depends on the size of the curve.

We commit to the scalar $v$ by creating the curve point $\text{Com}(v) = G * v$.

Because the discrete logarithm is hard, it is impossible to extract $v$ from $\text{Com}(v)$.

In [ ]:
def pedersen_attempt(v: Scalar) -> CurvePoint:
    return ONE_POINT * v

v = Scalar.random()
c1 = pedersen_attempt(v)
print(f"Commitment {c1}      opens to value {v}")

c2 = pedersen_attempt(v)
print(f"Commitment {c2} also opens to value {v}")

# Pedersen commitment (hiding)

There is a problem: Committing to the same value yields the same commitment. This reveals something about the contained value!

As a fix, we introduce a **random blinding factor** $r$.

To commit to the scalar $v$, we create the curve point $\text{Com}(v, r) = G * v + H * r$ where $r$ is random.

$H$ is a curve point that is known by all parties.

This resulting Pedersen commitment is **hiding**.

In [ ]:
h = Scalar(2)
H = ONE_POINT * h

def pedersen_hiding(v: Scalar) -> CurvePoint:
    r = Scalar.random()
    return ONE_POINT * v + H * r

v = Scalar.random()
c1 = pedersen_hiding(v)
print(f"Commitment {c1}      opens to value {v}")

c2 = pedersen_hiding(v)
print(f"Commitment {c2} also opens to value {v}")

# Hashing onto the curve

We can create curve points with random xy coordinate:

1. Generate random xy
2. Return point if curve equation is satisfied
3. Goto 1. otherwise

This is called **hashing onto the curve** because the random xy coordinates come from hash functions.

The resulting point $H$ is interesting because it is a **random point with unknown discrete logarithm!**

From our viewpoint, $H$ is independent from the one-point $G$. **No one knows** any linear relation between $G$ and $H$.

# Alternative one-points

Think of $H$ as a one-point in "another language".

We can iterate over the entire curve using $H$, but the order of points is completely different than for $G$.

We can iterate: $G * 1 = (4, 2)$, $G * 2 = (3, 3)$, $G * 3 = (1, 2)$, $\ldots$

Or we can iterate: $H * 1 = (2, 5)$, $H * 2 = (5, 4)$, $H * 3 = (4, 5)$, $\ldots$

The concrete coordinates depend on the curve and $G$ and $H$.

In [ ]:
puncto_uno, = CurvePoint.sample_greater_one(1)

print(f"G * 1 = {ONE_POINT * Scalar(1)}, G * 2 = {ONE_POINT * Scalar(2)}, G * 3 = {ONE_POINT * Scalar(3)}, ...")
print(f"H * 1 = {puncto_uno * Scalar(1)}, H * 2 = {puncto_uno * Scalar(2)}, H * 3 = {puncto_uno * Scalar(3)}, ...")

# Not binding

There is another problem: Peggy knows the blinding factor and the discrete logarithm of $H$:

$G * v + H * r = G * v + (G * h) * r$

She can open the same commitment to different values!

$\text{Com}(v, r) = G * v + (G * h) * r = G * (h * r) + (G * h) * \frac{v}{h} = \text{Com}(rh, \frac{v}{h})$

Here, Peggy changed the commitment from $v$ to $rh$. Many other shenanigans is possible.

In [ ]:
# Rerun this a couple of times

v = Scalar.random()
r = Scalar.random()
c1 = ONE_POINT * v + H * r
print(f"Commitment {c1} opens to value {v}")

hr = h * r
vh = v / h
c2 = ONE_POINT * hr + H * vh
print(f"Commitment {c2} opens to value {hr}")

assert c1 == c2

# Pedersen commitment (hiding and binding)

As a fix, we require that $H$ has an **unknown discrete logarithm**.

We cannot repeat our tricks from earlier.

To commit to the scalar $v$, we create the curve point $\text{Com}(v, r) = G * v + H * r$ where $r$ is random.

This resulting Pedersen commitment is hiding and **binding**.

In [ ]:
puncto_uno, = CurvePoint.sample_greater_one(1)

def pedersen_binding(v: Scalar) -> CurvePoint:
    r = Scalar.random()
    return ONE_POINT * v + puncto_uno * r

v = Scalar.random()
c1 = pedersen_binding(v)
print(f"Commitment {c1} opens to value {v}")

c2 = pedersen_binding(v)
print(f"Commitment {c2} also opens to value {v}")

# Pedersen commitment (multiple values)

We can commit to a **list of values** by using a **list of independent one-points**: $G_0$, $G_1$, $\ldots$, $G_n$, $H$

One of these can be the normal one-point and the rest must have an unknown discrete logarithm.

To commit to values $v_0$, $v_1$, $\ldots$, $v_n$, we create the **single curve point** $\text{Com}(v_0, v_1, \ldots, v_n; r) = G_0 * v_0 + G_1 * v_1 + \ldots + G_n * v_n + H * r$ where $r$ is random.

Notice how the list of values is compressed inside a single curve point! The commitment is extremely space-efficient (constant size). Opening the commitment requires revealing the list of values and the blinding factor (linear size).

The commitment is **hiding** because of the blinding factor.

The commitment is **binding** because we don't know the linear relations between the used one-points. Peggy is forced to use the same list of values plus blinding factor to restore the sum point that is the commitment.

In [ ]:
puncto_uno, premier_point, daiichi, = CurvePoint.sample_greater_one(3)

def pedersen_multiple(v0: Scalar, v1: Scalar, v2: Scalar) -> CurvePoint:
    r = Scalar.random()
    return ONE_POINT * r + puncto_uno * v0 + premier_point * v1 + daiichi * v2

v0, v1, v2 = Scalar.random(), Scalar.random(), Scalar.random()
c1 = pedersen_multiple(v0, v1, v2)
print(f"Commitment {c1}      opens to values {v0}, {v1}, {v2}")

c2 = pedersen_multiple(v0, v1, v2)
print(f"Commitment {c2} also opens to values {v0}, {v1}, {v2}")